In [176]:
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")

#BiocManager::install("treeio")
#BiocManager::install("ggtree")
#BiocManager::install("Biostrings")
#BiocManager::install("XVector")

#install.packages("tidyverse")
#install.packages("ggplot2")
#install.packages("ggstance")
#install.packages("ggmsa")
#install.packages("seqmagick")
#install.packages("shiny")
#install.packages("DT")
#install.packages("cowplot")
#install.packages("ggplotify")
#install.packages("shinydashboard")
#install.packages("shinydashboardPlus")
#install.packages("fontawesome")
#install.packages("UpSetR")
#install.packages("ggvenn")
###install.packages("plotly")
#install.packages("shinyjs")
install.packages("echarts4r")
###install.packages("rbokeh")
###install.packages("ggiraph")
###install.packages("VennDiagram")

also installing the dependencies ‘countrycode’, ‘corrplot’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [178]:
library(treeio)
library(ggtree)
library(tidyverse)
library(ggplot2)
library(ggstance)
library(Biostrings)
library(ggmsa)
library(seqmagick)
library(XVector)
library(gtable)
library(grid)
library(shiny)
library(DT)
library(cowplot)
library(ggplotify)
library(shinydashboard)
library(shinydashboardPlus)
library(fontawesome)
library(UpSetR)
library(ggvenn)
###library(plotly)
library(shinyjs)
library(echarts4r)
###library(ggiraph)
###library(rbokeh)
###library(VennDiagram)
###

## Ok so let's build a dashboard web app which displays some trees
## overview: 4 parts - preprocessing, ui, server, app calling

### preprocessing

In [ ]:
## define my two classes - defining the A2TEA HYPOTHESES object ;D
# class for the expanded_OG - containing all different types of data we have on it
setClass("expanded_OG", slots=list(genes="spec_tbl_df", 
                                  fasta_files="list", 
                                  msa="AAMultipleAlignment", 
                                  tree="phylo"))

# class for the hypotheses
setClass("hypothesis", slots=list(description="character", 
                                  number="character",
                                  expanded_in ="character", 
                                  compared_to="character", 
                                  expanded_OGs="list",
                                  species_tree="phylo"))


#load the A2TEA input
#load("/home/tyll/Desktop/PhD/A2TEA/shiny/example_trees/A2TEA_finished.RData")

## ui

In [340]:
header <- shinydashboardPlus::dashboardHeader(
  title = "A2TEA",
  tags$li(
      class = "dropdown",
# the following CSS keeps the header nice looking and on one line
# the CSS at the beginning of the sidebar definitons is also required for this
# sizes of individual elements and distances are important however the most important paramter for this to work is "height"
# this is set to 60px in all elements and ensures nice design 
## I'll probably have to adapt this a bit if/when I have a nice logo for A2TEA
## e.g. a hex design - https://github.com/GuangchuangYu/hexSticker
            tags$style(".main-header {max-height: 60px;}"),
            tags$style(".main-header .logo {height: 60px;font-size:26px;font-weight:bold;padding:10px;}"),
            tags$style(".sidebar-toggle {height: 60px; padding-top: 12px !important;font-size: 26px;}"),
            tags$style(".navbar {min-height:60px !important}"),
            tags$style(".form-control {padding: 16px 10px;}")),
      # add a github button in the header - to the repo or my profile
      tags$li(a(onclick = "onclick = window.open('https://github.com/tgstoecker')",
                                    href = NULL,
                                    icon("github"),
                                    title = "GitHub",
                                    style = "cursor: pointer; font-size: 32px; height: 60px;"),
                                    class = "dropdown"),
          # add a twitter button in the header - to my account
      tags$li(a(onclick = "onclick = window.open('https://twitter.com/tgstoecker')",
                                    href = NULL,
                                    icon("twitter"),
                                    title = "Twitter",
                                    style = "cursor: pointer; font-size: 32px; height: 60px;"),
                                    class = "dropdown")

)


sidebar <- dashboardSidebar(
  # call useShinyjs() in order to use it!
  useShinyjs(),
#  actionButton(inputId = "toggle", "Hide/show plot"),
  # as explained in the header creation - necessary CSS to guarantee complete vertical alignment
  tags$style(".left-side, .main-sidebar {padding-top: 60px}"),
  # this code hides the upload and select panels when the the sidebar is toggled off
  # it keeps the sidebar tabs though ;D
  # this is achieved by specifically referencing (and thus hiding) ".form-group"
  # https://www.w3schools.com/css/css_display_visibility.asp
  tags$style(".sidebar-collapse .form-group {display:none;}"),
  tags$style(".sidebar-collapse .logo {display:none;}"),
  # Add input of A2TE.RData object
  fileInput(inputId = "A2TEA",
                label = "Upload A2TEA.RData file:",
                multiple = FALSE,
                accept = ".RData"),
  # Add selectors for Hypothesis & HOG
  uiOutput('select_Hypothesis'),
#  uiOutput('select_HOG'),
  sidebarMenu(
    menuItem("General", tabName = "general", icon = icon("th")),
    menuItem("TEA analysis", icon = icon("th"), tabName = "tea"), #badgeLabel = "new", badgeColor = "green")
    menuItem("Differential expression", icon = icon("th"), tabName = "deg"),
    menuItem("Gene set enrichment", icon = icon("th"), tabName = "gsea")
  )

)

body <- dashboardBody(
  # call useShinyjs() in order to use it!
  useShinyjs(),
    
  tabItems(
    # first tab: general information; species plot, upsetR/venn diagramme
    tabItem(tabName = "general",
            h2("General information on the experiment"),
            # row1
            fluidRow(
              # Column 1
              column(width = 7, 
                     infoBox(
                       width = NULL,
                       title = "Inferred general stats",
                       subtitle = "Lorem ipsum blablabla",
                       fill = TRUE,
                       color = "yellow"
                     )
                  ),
              # Column 2
              column(width = 5,
                # in order to use shinyjs on box elements - wrap them in div() expressions
                # give the divs their own ids! in order to use shinyjs on the box
                shinyjs::hidden(
                  div(id = "id_speciesPlot_outer",
                  box(
                    id = "id_speciesPlot",
                    background ="yellow",
                    gradient = TRUE,
                    width = NULL,
                    title = "Phylogenetic Tree - Orthofinder/STAG/STRIDE",
                    plotOutput('speciesPlot')
                  )
                  )
                )
              )
            ),
            
            # row 2
            fluidRow(
              # Column 1
              column(width = 3,
                     shinyjs::hidden(
                       div(id = "id_vennPlot_outer",
                         box(
                           background ="yellow",
                           gradient = TRUE,
                           width = NULL,
                           title = "VennDiagram - Orthologous Groups",
                           plotOutput('vennPlot')
                         )
                       )
                     )  
                    ),
              # Column 2
              column(width = 4,
                shinyjs::hidden(
                  div(id = "id_upsetRPlot_outer",
                    box(
                      background ="yellow",
                      gradient = TRUE,
                      width = NULL,
                      title = "UpsetR Plot - Orthologous Groups",
                      plotOutput('upsetRPlot')
                    )
                  )
                )
              ),
              # Column 3
              column(width = 5)
            )
           ),
      
    # second tab: Trait-specific evolutionary adaptation analysis
    tabItem(tabName = "tea",
            h2("Trait-specific evolutionary adaption analysis"),
            uiOutput('select_HOG'),
            # select tree layout
            selectInput(
              inputId = "tree_layout",
              label = "Tree layout",
              choices = c('rectangular', 'dendrogram', 'slanted', 'ellipse', 'roundrect', 'fan', 'circular', 'inward_circular', 'radial', 'equal_angle', 'daylight', 'ape')
            ),
            # turn off/on branch_length option
            selectInput(
              inputId = "branch_length",
              label = "Branch length",
              choices = c('branch.length', 'none')
            ),
            # turn off/on branch_length option
            sliderInput(min = 0.1, max = 2.5, value = 1.0,
              inputId = "tree_size",
              label = "Tree Size"
            ),
            numericInput(inputId = "tree_height", "height", value = 1200),
#            sliderInput(inputId = "tree_width", "width", min = 100, max = 500, value = 250),
            selectInput(inputId = "tree_plot_legend_pos", "Legend position", choices = c("left", "top", "right", "bottom", "none")),
            # first row
            fluidRow(
              # row 1 - column 1
              column(width = 4, 
              ),
              # row 1 - column 2
              column(width = 8, 
                shinyjs::hidden(
                  div(id = "id_treePlot_outer",
                  # https://stackoverflow.com/questions/49129194/r-shiny-dynamic-box-height
                  # height: XX, where XX can be any CSS unit (I personally prefer viewpoint ratios, e.g. height: 33vh will make a box as high as one third of a screen, regardless of screen resolution), sets the height of your box;
                  # overflow-y: auto adds a vertical scrollbar if required.
                  # With this approach, when a user resizes the screen all boxes maintain their equal heights.
                    box(style = "height: 1000px; overflow-y: auto; overflow-x: auto",
                      background ="yellow",
                      gradient = TRUE,
                      width = NULL,
                      title = "Tree Plot - Orthologous Group",
                      plotOutput('treePlot')
                      )
                    )                  
                )
              )
            ),
#            plotOutput('treePlot')
           ),
    tabItem(tabName = "deg",
            h2("Differential expression analysis"),
           ),
    tabItem(tabName = "gsea",
            h2("Gene-set enrichment analysis")
           )
  )
)

ui <- shinydashboardPlus::dashboardPage(
        skin = "yellow",
        header = header,
        # sidebar should close completely
        sidebar = sidebar,
        body = body
      )

In [320]:
?ggtree

In [ ]:
server <- function(input, output) {
  
  # change maximum file upload size = default 100Mb
  options(shiny.maxRequestSize=100*1024^2)

  ## event reactive expression for loading the data once uploaded by the user
  # hypotheses.tsv (simple df with)
  hypotheses_tsv <- eventReactive(input$A2TEA, {
    t = new.env()
    load(input$A2TEA$datapath, envir = t)
    get("hypotheses", envir=t)
  })
  # core A2TEA object
  HYPOTHESES.a2tea <- eventReactive(input$A2TEA, {
    t = new.env()
    load(input$A2TEA$datapath, envir = t)
    get("HYPOTHESES.a2tea", envir=t)
  })
  # Gene level diff. exp. table
  HOG_DE.a2tea <- eventReactive(input$A2TEA, {
    t = new.env()
    load(input$A2TEA$datapath, envir = t)
    get("HOG_DE.a2tea", envir=t)
  })
  # List of HOGs
  HOG_level_list <- eventReactive(input$A2TEA, {
    t = new.env()
    load(input$A2TEA$datapath, envir = t)
    get("HOG_level_list", envir=t)
  })
    

  ## observe upload of data and only then show boxes in which the plots are displayed
  observeEvent(input$A2TEA, {
    shinyjs::showElement("id_speciesPlot_outer", anim = TRUE, animType = "slide", time = 1)
    shinyjs::showElement("id_vennPlot_outer", anim = TRUE, animType = "slide", time = 1)
    shinyjs::showElement("id_upsetRPlot_outer", anim = TRUE, animType = "slide", time = 1)
    # TEA analysis tab
    shinyjs::showElement("id_treePlot_outer", anim = TRUE, animType = "slide", time = 1)
  })
    
  ## rendering of UI elements for the sidebar
  # choice of hypothesis to be displayed
    output$select_Hypothesis = renderUI({
      selectInput(inputId = 'select_Hypothesis_server', 
                  label = 'Select Hypothesis:', 
                  choices = hypotheses_tsv()$name
                 )
    })
  
  # a dropdown list of the expanded HOGs of the current hypothesis
  # -> observe choice of HOG; e.g. for building trees
  # first create reactive list of all exp. HOGs 
  hypothesisExpOGs <- reactive({
    # require the input$select_Hypothesis_server to be present, since otherwise for a brief second an error is thrown
    req(input$select_Hypothesis_server)
    vars <- all.vars(
      parse(
        text = names(HYPOTHESES.a2tea()[[hypotheses_tsv()$hypothesis[hypotheses_tsv()$name==input$select_Hypothesis_server]]]@expanded_OGs)
      )
    )    
    vars <- as.list(vars)
    return(vars)    
  })
    
  output$select_HOG = renderUI({     
    selectInput('select_HOG_server', 'Select HOG', hypothesisExpOGs())
  })


########################
# General (start) page
########################

  # creating the species tree plot
  # make renderUI element reactive
  # access to element based on name in first element selectInput('')
  output$speciesPlot <- renderPlot({
    req(input$select_Hypothesis_server)
    ggtree(HYPOTHESES.a2tea()[[hypotheses_tsv()$hypothesis[hypotheses_tsv()$name==input$select_Hypothesis_server]]]@species_tree)
  })

  ## Creating upsetR and venn plots
  # based on hypothesis/ when it is changed, create specific list with incl. genotypes/species
  genotypeList <- eventReactive(input$select_Hypothesis_server, {
      
    workset <- HOG_level_list()[[hypotheses_tsv()$hypothesis[hypotheses_tsv()$name==input$select_Hypothesis_server]]] %>% 
                 select(HOG, ends_with("_total")) 
    # add this point we can easily compute the number of species/ecotypes etc. 
    # #species = (columns of current workset - 1) 
    species_num <- ncol(workset) - 1
    # change 0 to NAs since we can more easily filter on that
    is.na(workset) <- workset==0
    # remove '_total' from column names
    #str_remove(colnames(workset), "_total")
    workset <- workset %>% setNames(names(workset) %>% stringr::str_replace("_total",""))
    
    genotypeList <- list()
    for (j in 1:species_num) {
      assign(paste0("Genotype", j), as.vector(simplify_all(workset[!is.na(workset[j+1]),"HOG"])[[1]]))
      subList <- list(get(paste0("Genotype", j)))
      names(subList) <- colnames(workset)[j+1]
      genotypeList <- c(genotypeList, subList)
    }
    # return genotypeList
    genotypeList
  })

  # create Venn diagram
  output$vennPlot <- renderPlot({
    ggvenn(genotypeList(), 
           stroke_linetype = 2, 
           text_size = 5, 
           set_name_color = "navy",
           stroke_color = "yellow3",
           stroke_size = 0.8, 
           set_name_size = 6, 
           show_percentage = TRUE, 
           fill_color = c("yellow", "purple", "papayawhip", "red4")
          )
  })
    
  # create upsetR plot 
  output$upsetRPlot <- renderPlot({
    req(input$select_Hypothesis_server)
    upset(fromList(genotypeList()), 
          order.by = "freq",
          empty.intersections = "on",
          point.size = 3.5, 
          line.size = 2, 
          mainbar.y.label = "HOG Intersections", 
          text.scale = c(2, 1.3, 1, 1, 2, 2),
          main.bar.color = "navy",
          matrix.color = "navy",
          sets.bar.color = "navy"
         )
    })
        
    
      
########################
# Tea analysis 
########################
  # read in tree choice reactively
  # using eventReactive() and requiring the input$select_HOG_server to change leads to this specifically being the trigger
  # with only the basic reactive() all upstream events influence this and this leads to warning messages;
  # specifically the class of the tree object isn't found immediately
  # since select_HOG_server is dependent on the reactive select_Hypothesis_server we circumvent this here
  # changing HOG no problem, changing Hypothesis sets HOG to index 1 and is then evaluated -> no warnings and a clean App
  tree_choice <- eventReactive(input$select_HOG_server, {
    HYPOTHESES.a2tea()[[hypotheses_tsv()$hypothesis[hypotheses_tsv()$name==input$select_Hypothesis_server]]]@expanded_OGs[[input$select_HOG_server]]@tree
  })
    
    

  # creating the tree expansion plot
  # make renderUI element reactive
  # access to element based on name in first element selectInput('')
  #output$
    output$treePlot <- renderPlot(
#    width = function() input$tree_width,
    height = function() input$tree_height,
                                {
    #    width = "auto",
    req(input$select_Hypothesis_server)
    req(input$select_HOG_server)
    
    tree <- tree_choice() 
    info <- HOG_DE.a2tea()
      #ggtree(tree)
    
    # rlang package has the function is_empty()
    # with it we can test for "character(0)"
    el <- list()

    for (i in 1:length(tree$tip.label)) {
        if (is_empty(info$HOG[info$gene == tree$tip.label[i]]) == FALSE) {
           first <- info$HOG[info$gene == tree$tip.label[i]]
           el <- c(el, list(filler = tree$tip.label[i]))
           names(el)[i] <- first
        }
        else {
            el <- c(el, list(singleton = tree$tip.label[i]))      
        }
    }

    # reduce the list to unique tags
    el_split <- sapply(unique(names(el)), function(x) unname(unlist(el[names(el)==x])), simplify=FALSE)

    p_OTU <- ggtree(tree, 
                    layout = input$tree_layout,
                    branch.length = input$branch_length,
                    size = input$tree_size)
  
    p <- groupOTU(p_OTU, el_split, 'HOG') + 
                       aes(color=HOG) +
                       geom_tiplab(aes(color=HOG)) +
#                       theme(legend.position="right") +
                       coord_cartesian(clip = 'off') + 
                       theme_tree2(plot.margin=margin(6, 120, 6, 6), legend.position= input$tree_plot_legend_pos)
                       #scale_color_manual(values=cols,
                       #                   na.translate=TRUE,
                       #                   na.value = "black") +
                       #geom_tippoint(aes(color=HOG), align=F, offset = 0.3, alpha=1)
                       #geom_tiplab("ddkdkd")
                       
    return(p)
    #height = exprToFunction(as.numeric(input$dimension[2]) *  1/ 2  * 1 / 3)
    #p + lims(x = c(0, max(p$data$x) * input$subtree_width_multiply))
    #p
                       
  })
    
                                    
                                    
                                    
####################
# server side row rendering test
###############################
#  output$row_render_test <- renderUI({
#    req(input$select_Hypothesis_server)
#    req(input$select_HOG_server)
   # output <-  
      #fluidRow(
   #               column(width = 4,
   #                 numericInput(inputId = "tree_heightNull", "tree_heightNull", value = 3*300),
   #                     ),
   #               column(width = 8,
   #                 box(title = "dkdkd", 
   #                     gradient = TRUE, 
   #                     background ="yellow", 
   #                     width = NULL,
#                        plotOutput("treePlot", height = 3*300)
   #                      )
   #                     
#  })
                                    
}

                              
shinyApp(ui, server)


Listening on http://127.0.0.1:6268

Coordinate system already present. Adding new coordinate system, which will replace the existing one.

Coordinate system already present. Adding new coordinate system, which will replace the existing one.

Coordinate system already present. Adding new coordinate system, which will replace the existing one.

Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.

Coordinate system already present. Adding new coordinate system, which will replace the existing one.

Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.

Coordinate system already present. Adding new coordinate system, which will replace the existing one.

Coordinate system already present. Adding new coordinate system, which will replace the existing one.



In [332]:
?ggplot2::theme

In [ ]:
#load(file = "A2TEA_finished.RData")
#validObject(HYPOTHESES.a2tea$hypothesis_1@expanded_OGs$N0.HOG0000207@tree)

In [ ]:
# creating upsetR Plot
head(HOG_level_list[[1]])

workset <-  HOG_level_list[[1]]
# drop columns that are unnecessary
workset <- workset %>% select(HOG, ends_with("_total")) 
# add this point we can easily compute the number of species/ecotypes etc. 
# #species = (columns of current workset - 1) 
species_num <- ncol(workset) - 1
# change 0 to NAs since we can more easily filter on that
is.na(workset) <- workset==0
# remove '_total' from column names
#str_remove(colnames(workset), "_total")
workset <- workset %>% setNames(names(workset) %>% stringr::str_replace("_total",""))

head(workset)

genotypeList <- list()
for (j in 1:species_num) {
    assign(paste0("Genotype", j), as.vector(simplify_all(workset[!is.na(workset[j+1]),"HOG"])[[1]]))
    subList <- list(get(paste0("Genotype", j)))
    names(subList) <- colnames(workset)[j+1]
    genotypeList <- c(genotypeList, subList)
}

#head(genotypeList)

#using UpSetR
upsetOrtho <- upset(fromList(genotypeList), 
                    order.by = "freq",
                    empty.intersections = "on",
                    point.size = 3.5, 
                    line.size = 2, 
                    mainbar.y.label = "HOG Intersections", 
                    text.scale = c(2, 1.3, 1, 1, 2, 2),
                    main.bar.color = "navy",
                    matrix.color = "navy",
                    sets.bar.color = "navy"
                   )

upsetOrtho

ggvenn(genotypeList, stroke_linetype = 2, text_size = 5, set_name_color = "navy",
       stroke_color = "yellow3",
       stroke_size = 0.8, 
       set_name_size = 6, 
       show_percentage = TRUE, 
       fill_color = c("yellow", "purple", "papayawhip", "red4")
      )

## UpSetR playaround

In [ ]:
# drop columns that are unnecessary and get names of all species columns with total gene count per HOG
workset <- HOG_level_list[[1]] %>% select(ends_with("_total")) %>% names() 
# create copy of HOG_level_list[[1]] to be worked on
work_HOG_level_list <- HOG_level_list[[1]]

for (i in workset) {
    #print(i)
    species_name <- i %>% stringr::str_replace("_total","")
    #print(species_name)
    # create new columns for each species - 0,1 whether they have a gene in each HOG - this is needed or UpSetR
    work_HOG_level_list <- mutate(work_HOG_level_list, !!species_name := ifelse(
                                                      get(i) > 0, as.integer(1), as.integer(0)
                                                         )
         )
}
str(work_HOG_level_list)


#subset_work_HOG_level_list <- work_HOG_level_list %>% select(HOG, Arabidopsis_thaliana, Zea_mays, Hordeum_vulgare)
#str(subset_work_HOG_level_list)

# can't use tibble apparently...
# convert to dataframe first
df_work_HOG_level_list <- as.data.frame(work_HOG_level_list)
str(df_work_HOG_level_list)

df_work_HOG_level_list$Zea_mays_sigDE <- as.numeric(df_work_HOG_level_list$Zea_mays_sigDE)
df_work_HOG_level_list$Zea_mays_sigDE[is.na(df_work_HOG_level_list$Zea_mays_sigDE)] <- 0
#str(df_work_HOG_level_list)


#custom func to colour if sig DE genes from species A. thaliana
Myfunc <- function(row, num) {
    data <- (row["Hordeum_vulgare_sigDE"] > num)
}

#df_work_HOG_level_list %>% filter(!is.na(Zea_mays_sigDE)) %>% head()

upset(df_work_HOG_level_list,
      sets = c("Arabidopsis_thaliana", "Zea_mays", "Hordeum_vulgare"),
      order.by = "freq",
      empty.intersections = "on",
      point.size = 3.5, 
      line.size = 2, 
      mainbar.y.label = "HOG Intersections", 
      text.scale = c(2, 1.3, 1, 1, 2, 2),
      sets.bar.color = "navy",
      queries = list(list(query = Myfunc, params=list(3), color = "green", active = T))
     )

In [ ]:
head(df_work_HOG_level_list)
#names(df_work_HOG_level_list)
#hist(df_work_HOG_level_list$Zea_mays_sigDE)
#df_work_HOG_level_list[!is.na(df_work_HOG_level_list$Zea_mays_sigDE),]

In [ ]:
#movies <- read.csv(system.file("extdata", "movies.csv", package = "UpSetR"), 
#    header = T, sep = ";")
str(movies)

Myfunc <- function(row, rating) {
    data <- (row["AvgRating"] > rating)
}

Myfunc2 <- function(row, rating) {
    data <- (row["AvgRating"] < rating)
}

upset(movies, sets = c("Adventure", "Action", "SciFi"),
     queries = list(list(query = Myfunc, params = 2.5, color = "blue", active = T))#,
#                   list(query = Myfunc2, params = 2.5, color = "green", active = T))
    )

### ui

In [ ]:
# Define UI for tree app ----

#hypothesesList <- as.list(hypotheses$hypothesis)
#names(hypothesesList) <- hypotheses$name
#hypothesesList


# Allow specific errors to be displayed on screen, instead of displaying a generic error
options(shiny.sanitize.errors = FALSE)

ui <- pageWithSidebar(
  
  # App title ----
  headerPanel("Tree building"),
  
  # Sidebar panel for inputs ----
  sidebarPanel(
  
      fileInput(inputId = "A2TEA",
                label = "Upload A2TEA.RData file:",
                multiple = FALSE,
                accept = ".RData"),
      
        # Input: alternative way which leads to problems...
#      selectInput("select",
#                    label = h3("Select hypothesis:"),
#                    ""), 

      uiOutput('select_Hypothesis'),
      uiOutput('select_HOG')
      
  ),
  
  # Main panel for displaying outputs ----
  mainPanel(
      h3(textOutput("text")),
      plotOutput("speciesTree"),
      plotOutput("VennUpSet_plot"),
      DT::dataTableOutput("HOG_DE_DT"),
      DT::dataTableOutput("HOG_level"),
      plotOutput("recFacetPlot"),
  )
)

### server

In [ ]:
#increasing maximum file size for upload!
#https://stackoverflow.com/questions/18037737/how-to-change-maximum-upload-size-exceeded-restriction-in-shiny-and-save-user
options(shiny.maxRequestSize=50*1024^2)

myDataFrame <- data.frame(names=c(""))
#rm(myDataFrame)

server <- function(input, output, session) {
    

#read-in hypotheses.tsv reactively
    tsv <- reactive({
     inFile <- input$A2TEA
     if (is.null(inFile)) {
         d <- myDataFrame
    } else {
    t = new.env()
    load(input$A2TEA$datapath, envir = t)
    d <- get("hypotheses", envir=t)
     }
     d
   })  
    
#    observe({
#       updateSelectInput(session, "select",
#                         label = "Select Hypothesis:",
#                         choices = tsv()$name,
#                         selected = tsv()$name[1])
# })
    
    
#    output$hypothesesList <- renderText({
#        tsv()$name
#    })
    

    
    
#    tsv2 <- reactive({
#       vars <- all.vars(parse(text = names(HYPOTHESES()[[tsv()$hypothesis[tsv()$name==input$select]]]@expanded_OGs)))
#       vars <- as.list(vars)
#       return(vars)
    
    
    output$select_Hypothesis = renderUI({
      selectInput('select_Hypothesis2', 'Select Hypothesis:', tsv()$name)
    })

# also read-in the Ortho Venn/UpSet plots 
    Ortho_intersect_plots <- reactive({
     inFile <- input$A2TEA
     if (is.null(inFile)) {
         d <- myDataFrame
    } else {
    t5 = new.env()
    load(input$A2TEA$datapath, envir = t5)
    d <- get("Ortho_intersect_plots", envir = t5)
     }
     d
   }) 

    #plot the Ortho Venn/UpSet plots
    output$VennUpSet_plot <- renderPlot({
        Ortho_intersect_plots()[[tsv()$hypothesis[tsv()$name==input$select_Hypothesis2]]]
    })

    
# also read-in the DE genes table 
    HOG_DE <- reactive({
     inFile <- input$A2TEA
     if (is.null(inFile)) {
         d <- myDataFrame
    } else {
    t2 = new.env()
    load(input$A2TEA$datapath, envir = t2)
    d <- get("HOG_DE.a2tea", envir = t2)
     }
     d
   }) 


# also read-in custom A2TEA object
    HYPOTHESES <- reactive({
     inFile <- input$A2TEA
     if (is.null(inFile)) {
         d <- myDataFrame
    } else {
    t3 = new.env()
    load(input$A2TEA$datapath, envir = t3)
    d <- get("HYPOTHESES.a2tea", envir = t3)
     }
     d
   })  

# buld species tree
    #plot the Ortho Venn/UpSet plots
    output$speciesTree <- renderPlot({
       st <- HYPOTHESES()[[tsv()$hypothesis[tsv()$name==input$select_Hypothesis2]]]@species_tree
       ggtree(st, layout = 'rectangular', branch.length='yes', size=1.5) +
           geom_tiplab(align=F, offset = 0.1, alpha=1)
            
    })
    
    
# also read-in hypothesis specific HOG level table - HOG_level_list
    HOG_level <- reactive({
     inFile <- input$A2TEA
     if (is.null(inFile)) {
         d <- myDataFrame
    } else {
    t4 = new.env()
    load(input$A2TEA$datapath, envir = t4)
    d <- get("HOG_level_list", envir = t4)
     }
     d
   })  
    
    
# normal table chokes on large size of df
# therefore usage of DT::renderDataTable
    output$HOG_DE_DT <- DT::renderDataTable({
        HOG_DE()
  })
    
    
#example access to hypothesis number - useful for access 
    hypothesisNUM <- reactive({
#       HYPOTHESES()[[tsv()$hypothesis[tsv()$name==input$select]]]@number
        HYPOTHESES()[[tsv()$hypothesis[tsv()$name==input$select_Hypothesis2]]]@number
    })
   
     output$text <- renderText({
        hypothesisNUM()
    })    
        
        

        
# I can make use of the hypothesis number previously created
    output$HOG_level <- DT::renderDataTable({
#        HOG_level()[[tsv()$hypothesis[tsv()$name==input$select]]]
                HOG_level()[[tsv()$hypothesis[tsv()$name==input$select_Hypothesis2]]]
  })
             
     
        


# observe choice of HOG; e.g. for building trees
# first create reactive list of all exp. HOGs 
# due to my custom object this needs the following trickery: 
# all.vars(parse(text = names(....
hypothesisExpOGs <- reactive({
#       vars <- all.vars(parse(text = names(HYPOTHESES()[[tsv()$hypothesis[tsv()$name==input$select]]]@expanded_OGs)))
       vars <- all.vars(parse(text = names(HYPOTHESES()[[tsv()$hypothesis[tsv()$name==input$select_Hypothesis2]]]@expanded_OGs)))    
       vars <- as.list(vars)
       return(vars)    
    })
    
# actually render the UI element - a dropdown list of the expanded HOGs of the current hypothesis
    output$select_HOG = renderUI({
      selectInput('select_HOG2', 'Select HOG', hypothesisExpOGs())
    })
    
# creating the rectangular A2TEA triple facet plot

# make renderUI element reactive
# access to element based on name in first element selectInput('')
    choice <- reactive({input$select_HOG2})
    
output$recFacetPlot <- renderPlot({

    #### building the rectangular facet plot - HOG tree, logFC & MSA
                     
#    tree <- HYPOTHESES()[[tsv()$hypothesis[tsv()$name==input$select]]]@expanded_OGs[[choice()]]@tree
    tree <- HYPOTHESES()[[tsv()$hypothesis[tsv()$name==input$select_Hypothesis2]]]@expanded_OGs[[choice()]]@tree
    info <- HOG_DE()


# extracted from colourblind - removed black... how to deal with NAs..
# also a problem if more than given colours are supposed to be displayed error....
# could perform check of number of HOGs and switch between an optimized colour scale with predefined colours
# to an open-ended/automatic colour scheme if set number is exceeded by choice of HOG
    cols <- c("#E69F00", "#56B4E9", "#009E73", "#F0E442", "#0072B2", "#D55E00", "#CC79A7", "lightblue", "darkred",
              "lightblue", "darkred", "pink", "cyan", "grey", "violet", "magenta", "gold", 
              "darkgreen", "darkblue")


# rlang package has the function is_empty()
# with it we can test for "character(0)"
    el <- list()

    for (i in 1:length(tree$tip.label)) {
        if (is_empty(info$HOG[info$gene == tree$tip.label[i]]) == FALSE) {
           first <- info$HOG[info$gene == tree$tip.label[i]]
           el <- c(el, list(filler = tree$tip.label[i]))
           names(el)[i] <- first
        }
        else {
            el <- c(el, list(singleton = tree$tip.label[i]))      
        }
    }

# reduce the list to unique tags
    el_split <- sapply(unique(names(el)), function(x) unname(unlist(el[names(el)==x])), simplify=FALSE)

    p_OTU <- ggtree(tree,
                    layout = 'rectangular',
                    branch.length='none',
                    size=1.5)

    p <- groupOTU(p_OTU, el_split, 'HOG') + 
                       aes(color=HOG) +
                       geom_tippoint(aes(color=HOG)) +
                       scale_color_manual(values=cols,
                                          na.translate=TRUE,
                                          na.value = "black") +
                       geom_tiplab(aes(color=HOG), 
                                   align=F, 
                                   offset = 0.3,
                                   alpha=1)
                                   

# this is such a convenient way to do this ;D + we also drop the columns we don't need
    d <- filter(info, gene %in% tree$tip.label) %>% 
             select(-c(species, baseMean, HOG, lfcSE, stat, pvalue, padj))

    lines=data.frame(y = c(-2,-1,1,2), .panel='log2FC')

                   
# xlim set x axis limits for only Tree panel
# xlim tree calculated as number of tips tips?
    p2 <- facet_plot(p + xlim_tree(16), 
                     panel = 'log2FC', 
                     data = d, 
                     geom = geom_barh, 
                     mapping = aes(x = log2FoldChange),
                     stat='identity',
                 #inherit.aes = TRUE,
                     color='firebrick') + 
                       theme_tree2(plot.margin=margin(0, 0, 0, 0)) + 
                       geom_vline(data=lines, 
                                  aes(xintercept=y), 
                                  linetype = "dashed")
                   
                   
# get the msa and assign it to x (for now...)
# set msa range of amino acids to be displayed - make this dynamcic!
#    x <- HYPOTHESES()[[tsv()$hypothesis[tsv()$name==input$select]]]@expanded_OGs[[choice()]]@msa
    x <- HYPOTHESES()[[tsv()$hypothesis[tsv()$name==input$select_Hypothesis2]]]@expanded_OGs[[choice()]]@msa
    data <- tidy_msa(x, start = 1, end = 598)


# use font to display amino acid code, eg "helvetical"
p3 <- p2 + geom_facet(geom = geom_msa, 
               data = data,  
               panel = 'msa',
               font = NULL, 
               color = "Clustal")
                   

    gt = ggplot_gtable(ggplot_build(p3))
#gt$layout$l[grep('background', gt$layout$name)]
#gt$widths[1] = 20*gt$widths[1] # in this case it was colmun 5 - double the width
    gt$widths[5] = 2*gt$widths[5] # in this case it was colmun 5 - double the width
    gt$widths[7] = 2*gt$widths[7] # in this case it was colmun 7 - double the width
    gt$widths[9] = 5*gt$widths[9] # in this case it was colmun 9 - double the width

# this option is nice for jupyter but I have to see what will be possible in shiny                   
    options(repr.plot.width=15, repr.plot.height=8)
    grid.draw(gt)
            
            
############ ending facet plot code ############
            
  })
        
}

### app calling

In [ ]:
shinyApp(ui, server)

### playground & working examples

In [ ]:
hypotheses$hypothesis[hypotheses$name=="Expanded in barley compared to maize"]

In [ ]:
saveRDS( data.frame(names=c("Jill","Jane","Megan")),"example_trees/myDataFrame.rds")

#### completely working mini application showcasing runtime loading of data and reactive dropdown

In [ ]:
myDataFrame <- data.frame(names=c("Tom","Dick","Harry"))

ui <- shinyUI(
    fluidPage(
        fileInput("file1", "Choose file to upload", accept = ".rds"),
        selectInput("myNames","Names", ""),
        tableOutput("contents")
    )
)

server <- function(input, output, session) {

    myData <- reactive({
        inFile <- input$file1
        if (is.null(inFile)) {
            d <- myDataFrame
        } else {
            d <- readRDS(inFile$datapath)
        }
        d
    })

    output$contents <- renderTable({
        myData()
    })

    observe({
         updateSelectInput(session, "myNames",
                           label = "myNames",
                           choices = myData()$names,
                           selected = myData()$names[1])
    })

}

shinyApp(ui, server)


#### shiny example of using RData input

In [ ]:
#The example allows you to upload .RData files.
#The approach with load and get allows you to assign the loaded data to a variable name of your choice.
#For the matter of the example being "standalone" I inserted the top section that stores 
#two vectors to your disk in order to load and plot them later.


# Define two datasets and store them to disk
x <- rnorm(100)
save(x, file = "x.RData")
#rm(x)
y <- rnorm(100, mean = 2)
save(y, file = "y.RData")
#rm(y)
# both
save(x, y, file = "z.RData")

# This function, borrowed from http://www.r-bloggers.com/safe-loading-of-rdata-files/, load the Rdata into a new environment to avoid side effects
LoadToEnvironment <- function(RData, env=new.env()) {
  load(RData, env)
  return(env)
}


# Define UI
ui <- shinyUI(fluidPage(
  titlePanel(".RData File Upload Test"),
  mainPanel(
    fileInput("file", label = ""),
    actionButton(inputId="plot","Plot"),
    plotOutput("hist"))
  )
)

# Define server logic
server <- shinyServer(function(input, output) {

  observeEvent(input$plot,{
    if ( is.null(input$file)) return(NULL)
    inFile <- input$file
    file <- inFile$datapath
    # load the file into new environment and get it from there
#    name <- load(file, envir = e)
#    data <- e[[name]]
    # this works but not really what I need
#    data <- e[[name[1]]]
    t = new.env()
    load(file, envir = t)
    data <- get("y", envir=t)

      
      
#    observeEvent(input$plot,{
#        if ( is.null(input$file)) return(NULL)
#        inFile <- input$file
#        file <- inFile$datapath
#    # Use a reactiveFileReader to read the file on change, and load the content into a new environment
#    env <- reactiveFileReader(1000, session, "z.RData", LoadToEnvironment)
#    # Access the first item in the new environment, assuming that the Rdata contains only 2 items which is a data frame
#    env()[[names(env())[2]]]
    
    
    # Plot the data
    output$hist <- renderPlot({
      hist(data)
    })
  })
})

# Run the application 
shinyApp(ui = ui, server = server)

In [ ]:
e = new.env()
name <- load("z.RData", envir = e)
#mget(c("y","x"), envir=e)
#get returns exactly the object
test <- get("y", envir=e)
str(test)
str(y)

In [ ]:
#load("example_trees/backup_old_RData//A2TEA_finished.RData")
#str(hypotheses$name)
#HOG_DE.a2tea
#HYPOTHESES.a2tea
#HOG_level_list[[1]][HOG_level_list[[1]]$expansion=="yes",]
#tt <- as.data.frame(names(HYPOTHESES.a2tea[[1]]@expanded_OGs))
#names(tt)
#names(tt)[names(tt) == "names(HYPOTHESES.a2tea[[1]]@expanded_OGs)"] <- "HOG"
#names(tt)
#tt$HOG
#tt[[1]]
#?base::as.data.frame


load("A2TEA_finished.RData")
#str(HOG_level_list)
#hypotheses
#HYPOTHESES.a2tea$hypothesis_1@description
#HYPOTHESES.a2tea()[[hypotheses_tsv()$hypothesis[hypotheses_tsv()$name==input$select_Hypothesis_server]]]@expanded_OGs


HYPOTHESES.a2tea[[hypotheses$hypothesis[hypotheses$name=="Expanded in Arabidopsis compared to Monocots"]]]@expanded_OGs

In [ ]:
HYPOTHESES.a2tea[[hypotheses$hypothesis[hypotheses$name=="Expanded in Arabidopsis compared to Monocots"]]]@number